In [1]:
import tensorflow as tf
tf.__version__

/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

'2.0.0-alpha0'

In [2]:
import pandas as pd

df = pd.read_csv('../data/isear_processed.csv',index_col=0, names=['text','emotion'], header=0)

In [3]:
df.head()

,text,emotion
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust


In [4]:
df['emotion'].value_counts()

anger      1096
disgust    1096
sadness    1096
shame      1096
fear       1095
joy        1094
guilt      1093
Name: emotion, dtype: int64

In [5]:
emotion_dict = {}
labels = df['emotion']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
print('Creating the emotion dict to encode the label')

Creating the emotion dict to encode the label


In [6]:
texts = df['text']

In [7]:
import numpy as np

encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

seq_length = 50

tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, maxlen=50)

Using TensorFlow backend.


In [9]:
embeddings_index = {}
with open('../data/embedding/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
#Preparing the embedding

MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

num_words = min(MAX_NUM_WORDS, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [12]:
import torch

train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train).type(torch.LongTensor), torch.from_numpy(np.array(y_train)).type(torch.LongTensor))
val_data = TensorDataset(torch.from_numpy(X_test).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor))

# dataloaders
batch_size = 64

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [14]:
from basic_transformer import Transformer

num_words = len(tokenizer.word_index) + 1
num_classes = len(labels.unique())

model = Transformer(100,4,1,num_words, seq_length,num_classes,True,embedding_matrix)

print(model)

Transformer(
  (token_embedding): Embedding(9064, 100)
  (position_embedding): Embedding(50, 100)
  (transfo_blocks): Sequential(
    (0): TransformerBlock(
      (self_attention): SelfAttention(
        (to_keys): Linear(in_features=100, out_features=400, bias=True)
        (to_queries): Linear(in_features=100, out_features=400, bias=True)
        (to_values): Linear(in_features=100, out_features=400, bias=True)
        (merge_heads): Linear(in_features=400, out_features=100, bias=True)
      )
      (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=100, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=100, bias=True)
      )
    )
  )
  (to_probs): Linear(in_features=100, out_features=7, bias=True)
)


In [15]:
import torch.nn as nn
from torch import optim
from torch.nn import functional as F

criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters())

In [16]:
from sklearn.metrics import accuracy_score

EPOCHS = 30
for e in range(EPOCHS):
    model.train()

    for text_sequences, targets in train_loader:

        opt.zero_grad()

        out = model(text_sequences)
        loss = criterion(out, targets)

        loss.backward()

        opt.step()

    cur_loss = loss.item()
        
    model.eval()
    
    val_loss = []
    y_pred = []
    y_true = []
    
    val_loss = []
    
    for text_sequences,targets in val_loader:
        with torch.no_grad():
            out = model(text_sequences)
            loss = criterion(out, targets)
            val_loss.append(loss.item())
            y_pred+=list(out.detach().numpy().argmax(axis=1))
            y_true+=list(targets.detach().numpy())
        
    model.train()
    
    print("Epoch: {}/{}  ".format(e+1, EPOCHS),
            "Loss: {:.6f}  ".format(cur_loss),
              "val_loss: {:.6f}".format(np.mean(val_loss)),
             "val_accuracy: {:.6f}".format(accuracy_score(y_true,y_pred)))

Epoch: 1/30   Loss: 1.915957   val_loss: 1.914337 val_accuracy: 0.208659
Epoch: 2/30   Loss: 1.831618   val_loss: 1.833256 val_accuracy: 0.321335
Epoch: 3/30   Loss: 1.828008   val_loss: 1.856948 val_accuracy: 0.275952
Epoch: 4/30   Loss: 1.769396   val_loss: 1.788203 val_accuracy: 0.361502
Epoch: 5/30   Loss: 1.747059   val_loss: 1.781728 val_accuracy: 0.362546
Epoch: 6/30   Loss: 1.766260   val_loss: 1.771320 val_accuracy: 0.369327
Epoch: 7/30   Loss: 1.789414   val_loss: 1.769282 val_accuracy: 0.383412
Epoch: 8/30   Loss: 1.749119   val_loss: 1.783310 val_accuracy: 0.362024
Epoch: 9/30   Loss: 1.574376   val_loss: 1.744958 val_accuracy: 0.400104
Epoch: 10/30   Loss: 1.623026   val_loss: 1.737799 val_accuracy: 0.416797
Epoch: 11/30   Loss: 1.578789   val_loss: 1.745431 val_accuracy: 0.402713
Epoch: 12/30   Loss: 1.635651   val_loss: 1.742455 val_accuracy: 0.403756
Epoch: 13/30   Loss: 1.681242   val_loss: 1.744960 val_accuracy: 0.405321
Epoch: 14/30   Loss: 1.728574   val_loss: 1.731